# Data prep

Este notebook é dedicado ao data prep e exportação de um csv com os dados trabalhados ('diabetic_data_df.csv') e as colunas utilizadas ('col2use.csv')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
# load the csv file
df = pd.read_csv('diabetic_data.csv')

## Algumas análises

In [3]:
print('Number of samples:',len(df))

Number of samples: 101766


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# count the number of rows for each type
df.groupby('readmitted').size()

In [ ]:
# discharge_disposition_id, tells us where the patient went after the hospitalization.
df.groupby('discharge_disposition_id').size()

In [ ]:
#visualização de 10 em 10 colunas
df[list(df.columns)[:10]].head()

In [ ]:
#visualização de 10 em 10 colunas
df[list(df.columns)[10:20]].head()

In [ ]:
#visualização de 10 em 10 colunas
df[list(df.columns)[20:30]].head()

In [ ]:
#visualização de 10 em 10 colunas
df[list(df.columns)[30:40]].head()

In [ ]:
#visualização de 10 em 10 colunas
df[list(df.columns)[40:]].head()

In [4]:
# Mostra os IDs unicos da base se forem (em quantidade) menores que 30, caso contrário diz quantos são
# for each column
for c in list(df.columns):
    
    # get a list of unique values
    n = df[c].unique()
    
    # if number of unique values is less than 30, print the values. Otherwise print the number of unique values
    if len(n)<30:
        print(c)
        print(n)
    else:
        print(c + ': ' +str(len(n)) + ' unique values')

encounter_id: 101766 unique values
patient_nbr: 71518 unique values
race
['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
gender
['Female' 'Male' 'Unknown/Invalid']
age
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
weight
['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
admission_type_id
[6 1 2 3 4 5 8 7]
discharge_disposition_id
[25  1  3  6  2  5 11  7 10  4 14 18  8 13 12 16 17 22 23  9 20 15 24 28
 19 27]
admission_source_id
[ 1  7  2  4  5  6 20  3 17  8  9 14 10 22 11 25 13]
time_in_hospital
[ 1  3  2  4  5 13 12  9  7 10  6 11  8 14]
payer_code
['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
medical_specialty: 73 unique values
num_lab_procedures: 118 unique values
num_procedures
[0 5 1 6 2 3 4]
num_medications: 75 unique values
number_outpatient: 39 unique values
number_emergency: 33 unique values
number_inpatient

## Trabalhando a base

In [5]:
# De acordo com IDs_mapping.csv os IDs 11,13,14,19,20,21 são relacionados a morte ou hospício. Remover esses IDs do modelo.
df = df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]

In [6]:
# Criando a coluna de output para classificação binária
df['OUTPUT_LABEL'] = (df.readmitted == '<30').astype('int')

In [7]:
# replace ? with nan
df = df.replace('?',np.nan)

In [8]:
# nas colunas 'race', 'player_code' e 'medical_specialty' estao faltando dados, preencher com 'UNK'
df['race'] = df['race'].fillna('UNK')
df['payer_code'] = df['payer_code'].fillna('UNK')
df['medical_specialty'] = df['medical_specialty'].fillna('UNK')

In [9]:
df.groupby('medical_specialty').size()

medical_specialty
AllergyandImmunology                        7
Anesthesiology                             12
Anesthesiology-Pediatric                   19
Cardiology                               5279
Cardiology-Pediatric                        7
DCPTEAM                                     5
Dentistry                                   4
Dermatology                                 1
Emergency/Trauma                         7419
Endocrinology                             119
Endocrinology-Metabolism                    8
Family/GeneralPractice                   7252
Gastroenterology                          538
Gynecology                                 58
Hematology                                 75
Hematology/Oncology                       187
Hospitalist                                57
InfectiousDiseases                         37
InternalMedicine                        14237
Nephrology                               1539
Neurology                                 201
Neurophysiology 

In [10]:
# Sobre a coluna 'medical_specialty', pega os top 10 mais recorrentes e cria uma nova coluna contendo somente estes top 10
# os outros dados que não estão dentro dos top 10 são classificados como 'Other'
top_10 = ['UNK','InternalMedicine','Emergency/Trauma',\
          'Family/GeneralPractice', 'Cardiology','Surgery-General' ,\
          'Nephrology','Orthopedics',\
          'Orthopedics-Reconstructive','Radiologist']

# make a new column with duplicated data
df['med_spec'] = df['medical_specialty'].copy()

# replace all specialties not in top 10 with 'Other' category
df.loc[~df.med_spec.isin(top_10),'med_spec'] = 'Other'

### Encoding categorical data

A) Categorical columns:
   race, gender, max, max_glu_serum, A1Cresult,
   metformin, repaglinide, nateglinide, chlorpropamide,
   glimepiride, acetohexamide, glipizide, glyburide, tolbutamide,
   pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone,
   tolazamide, insulin,
   glyburide-metformin, glipizide-metformin,
   glimepiride-pioglitazone, metformin-rosiglitazone,
   metformin-pioglitazone, change, diabetesMed,payer_code,
   admission_type_id, discharge_disposition_id, admission_source_id

B) What is the unique labels for each column?

In [11]:
cols_cat = ['race', 'gender', 
       'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed','payer_code', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id']

for c in list(cols_cat):
    
    # get a list of unique values
    n = df[c].unique()
    
    # if number of unique values is less than 30, print the values. Otherwise print the number of unique values
    if len(n)<30:
        print('\033[94m {} \033[0m'.format(c))
        print(n)

 race 
['Caucasian' 'AfricanAmerican' 'UNK' 'Other' 'Asian' 'Hispanic']
 gender 
['Female' 'Male' 'Unknown/Invalid']
 max_glu_serum 
['None' '>300' 'Norm' '>200']
 A1Cresult 
['None' '>7' '>8' 'Norm']
 metformin 
['No' 'Steady' 'Up' 'Down']
 repaglinide 
['No' 'Up' 'Steady' 'Down']
 nateglinide 
['No' 'Steady' 'Down' 'Up']
 chlorpropamide 
['No' 'Steady' 'Down' 'Up']
 glimepiride 
['No' 'Steady' 'Down' 'Up']
 acetohexamide 
['No' 'Steady']
 glipizide 
['No' 'Steady' 'Up' 'Down']
 glyburide 
['No' 'Steady' 'Up' 'Down']
 tolbutamide 
['No' 'Steady']
 pioglitazone 
['No' 'Steady' 'Up' 'Down']
 rosiglitazone 
['No' 'Steady' 'Up' 'Down']
 acarbose 
['No' 'Steady' 'Up' 'Down']
 miglitol 
['No' 'Steady' 'Down' 'Up']
 troglitazone 
['No' 'Steady']
 tolazamide 
['No' 'Steady' 'Up']
 insulin 
['No' 'Up' 'Steady' 'Down']
 glyburide-metformin 
['No' 'Steady' 'Down' 'Up']
 glipizide-metformin 
['No' 'Steady']
 glimepiride-pioglitazone 
['No' 'Steady']
 metformin-rosiglitazone 
['No' 'Steady']
 metf

C) Separate the categorical columns into nominal, ordinal and numeric

In [12]:
cols_cat_nominal = ['race', 'gender', 'change', 'diabetesMed', 'med_spec'] # 'payer_code',
cols_cat_ordinal = ['max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
cols_cat_num = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']

D) Nominal data is encoded with one hot encoding using get_dummies

In [13]:
#one-hot for nominal data
df_cat_nominal = pd.get_dummies(df[cols_cat_nominal])
df_cat_nominal.head(5)

,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UNK,gender_Female,gender_Male,gender_Unknown/Invalid,change_Ch,...,med_spec_Emergency/Trauma,med_spec_Family/GeneralPractice,med_spec_InternalMedicine,med_spec_Nephrology,med_spec_Orthopedics,med_spec_Orthopedics-Reconstructive,med_spec_Other,med_spec_Radiologist,med_spec_Surgery-General,med_spec_UNK
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1


In [14]:
#one-hot for numeric data
df[cols_cat_num] = df[cols_cat_num].astype('str')
df_cat_numeric = pd.get_dummies(df[cols_cat_num])
df_cat_numeric.head(5)

,admission_type_id_1,admission_type_id_2,admission_type_id_3,admission_type_id_4,admission_type_id_5,admission_type_id_6,admission_type_id_7,admission_type_id_8,discharge_disposition_id_1,discharge_disposition_id_10,...,admission_source_id_20,admission_source_id_22,admission_source_id_25,admission_source_id_3,admission_source_id_4,admission_source_id_5,admission_source_id_6,admission_source_id_7,admission_source_id_8,admission_source_id_9
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
#numeric encoder for ordinal data

# ['No', 'Down', 'Steady', 'Up']  -> [1, 2, 3, 4]
# cols_cat_1 = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',  
#              'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 
#              'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 
#              'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 
#              'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
# ['None', 'Norm', '>200', '>300']-> [1, 2, 3, 4]
# cols_cat_2 = ['max_glu_serum'] 
# ['None,' 'Norm',  '>7', '>8'] -> [1, 2, 3, 4]
scale_mapper = {'No':1, 
                  'Down':2,
                  'Steady':3,
                  'Up':4,
                  'None':1,
                  'Norm':2,
                  '>200':3,
                  '>300':4,
                  '>7':3,
                  '>8':4}
df_cat_ord = df[cols_cat_ordinal].replace(scale_mapper)

for c in list(cols_cat_ordinal):
    
    # get a list of unique values
    n = df[c].unique()
    n2 = df_cat_ord[c].unique()
    
    # if number of unique values is less than 30, print the values. Otherwise print the number of unique values
    if len(n)<30:
        print('\033[94m {} \033[0m'.format(c))
        print('{} | {} '.format(n, n2))

 max_glu_serum 
['None' '>300' 'Norm' '>200'] | [1 4 2 3] 
 A1Cresult 
['None' '>7' '>8' 'Norm'] | [1 3 4 2] 
 metformin 
['No' 'Steady' 'Up' 'Down'] | [1 3 4 2] 
 repaglinide 
['No' 'Up' 'Steady' 'Down'] | [1 4 3 2] 
 nateglinide 
['No' 'Steady' 'Down' 'Up'] | [1 3 2 4] 
 chlorpropamide 
['No' 'Steady' 'Down' 'Up'] | [1 3 2 4] 
 glimepiride 
['No' 'Steady' 'Down' 'Up'] | [1 3 2 4] 
 acetohexamide 
['No' 'Steady'] | [1 3] 
 glipizide 
['No' 'Steady' 'Up' 'Down'] | [1 3 4 2] 
 glyburide 
['No' 'Steady' 'Up' 'Down'] | [1 3 4 2] 
 tolbutamide 
['No' 'Steady'] | [1 3] 
 pioglitazone 
['No' 'Steady' 'Up' 'Down'] | [1 3 4 2] 
 rosiglitazone 
['No' 'Steady' 'Up' 'Down'] | [1 3 4 2] 
 acarbose 
['No' 'Steady' 'Up' 'Down'] | [1 3 4 2] 
 miglitol 
['No' 'Steady' 'Down' 'Up'] | [1 3 2 4] 
 troglitazone 
['No' 'Steady'] | [1 3] 
 tolazamide 
['No' 'Steady' 'Up'] | [1 3 4] 
 insulin 
['No' 'Up' 'Steady' 'Down'] | [1 4 3 2] 
 glyburide-metformin 
['No' 'Steady' 'Down' 'Up'] | [1 3 2 4] 
 glipizide-m

In [16]:
df_cat = pd.concat([df_cat_nominal, df_cat_numeric], axis = 1)
df = pd.concat([df,df_cat], axis = 1)
df[cols_cat_ordinal] = df_cat_ord[cols_cat_ordinal]
# salva as colunas de dados categóricos em 'cols_all_cat'
cols_all_cat = list(df_cat.columns) + cols_cat_ordinal

In [17]:
# transformação das colunas de idade e peso de categóricas para numéricas
age_id = {'[0-10)':0, 
          '[10-20)':10, 
          '[20-30)':20, 
          '[30-40)':30, 
          '[40-50)':40, 
          '[50-60)':50,
          '[60-70)':60, 
          '[70-80)':70, 
          '[80-90)':80, 
          '[90-100)':90}
df['age_group'] = df.age.replace(age_id)

df['has_weight'] = df.weight.notnull().astype('int')

# salva as colunas 'age_group' e 'has_weight'
cols_extra = ['age_group','has_weight']

In [18]:
cols_num = ['time_in_hospital','num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient','number_diagnoses']

In [19]:
# cria o df_data que é o dataframe final com os dados utilizados
col2use = cols_num + cols_all_cat + cols_extra
col2use_df = pd.DataFrame(list(col2use), columns=['col2use'])
col2use_df.to_csv('new_col2use.csv')

df_data = df[col2use + ['OUTPUT_LABEL']]

In [20]:
# exporta csv do dataframe final
df_data.to_csv(path_or_buf="new_diabetic_data_df.csv", index=False)

In [ ]:
df_data.head()

In [21]:
df_data[cols_cat_ordinal]

,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,...,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,4,1,1,1,1,1
2,1,1,1,1,1,1,1,1,3,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,4,1,1,1,1,1
4,1,1,1,1,1,1,1,1,3,1,...,1,1,1,1,3,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,3,1,1,1,1,1
6,1,1,3,1,1,1,3,1,1,1,...,1,1,1,1,3,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,3,...,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,3,1,...,1,1,1,1,3,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,3,1,1,1,1,1
